In [2]:
from matplotlib import pyplot as plt
import pandas
import os
from datetime import datetime
from getpass import getpass
import uuid
import random
from cognite.client.experimental import CogniteClient
from cognite.client.data_classes import Asset, Relationship, TimeSeries
from cognite.client.exceptions import *

In [3]:
project = input("Enter project name: ")

Enter project name:  caelyn-first-project


In [4]:
base_url = input("Enter base url for project (leave blank to use greenfield)")
if (base_url == ""):
    base_url = "https://greenfield.cognitedata.com"

print("Using base url: ", base_url)

Enter base url for project (leave blank to use greenfield) 


Using base url:  https://greenfield.cognitedata.com


In [5]:
api_key_request = "Enter API-KEY for % s :" % project
api_key = getpass(api_key_request)

Enter API-KEY for caelyn-first-project : ················································


In [6]:
client_name = "Time Stitching Demo"
client = CogniteClient(project=project,api_key=api_key,client_name=client_name, base_url=base_url)

In [32]:
source = "time_stitching_demo"
widget_heat_sensor_external_id = "widgetHeatSensor:" + str(uuid.uuid4())
widget_heat_sensor = Asset(
    external_id=widget_heat_sensor_external_id,
    name="widgetHeatSensor",
    description="A heat sensor for a widget",
    source=source
)
sensor_instance_1_external_id = "heatSensor:" + str(uuid.uuid4())
sensor_instance_1 = Asset(
    external_id=sensor_instance_1_external_id,
    parent_external_id = widget_heat_sensor_external_id,
    name="heatSensor",
    description="A heat sensor unit",
    source=source
)
sensor_instance_2_external_id = "heatSensor:" + str(uuid.uuid4())
sensor_instance_2 = Asset(
    external_id=sensor_instance_2_external_id,
    parent_external_id=widget_heat_sensor_external_id,
    name="heatSensor",
    description="A heat sensor unit",
    source=source
)
assets = client.assets.create([widget_heat_sensor, sensor_instance_1, sensor_instance_2]).to_pandas()
assets

,externalId,name,description,metadata,source,id,createdTime,lastUpdatedTime,rootId,parentId,parentExternalId
0,widgetHeatSensor:229fd248-51b8-40a5-9f28-0793b...,widgetHeatSensor,A heat sensor for a widget,{},time_stitching_demo,7736683234368567,1586183545674,1586183545674,7736683234368567,<NA>,NaN
1,heatSensor:910db54d-dde6-452a-8cf0-8c7e887a28e2,heatSensor,A heat sensor unit,{},time_stitching_demo,7418793056223224,1586183545674,1586183545674,7736683234368567,7736683234368567,widgetHeatSensor:229fd248-51b8-40a5-9f28-0793b...
2,heatSensor:3b9eb1d1-1e49-41d3-8e97-8dcd5f6d4aee,heatSensor,A heat sensor unit,{},time_stitching_demo,6842794385662259,1586183545674,1586183545674,7736683234368567,7736683234368567,widgetHeatSensor:229fd248-51b8-40a5-9f28-0793b...


In [34]:
sensor_instance_1_id = assets.loc[assets["externalId"] == sensor_instance_1_external_id].iloc[0]["id"]
sensor_instance_2_id = assets.loc[assets["externalId"] == sensor_instance_2_external_id].iloc[0]["id"]

sensor_1_timeseries_external_id = sensor_instance_1_external_id + ":timeseries"
sensor_1_timeseries = TimeSeries(
    external_id=sensor_1_timeseries_external_id,
    asset_id=sensor_instance_1_id,
    name="heatSensorTimeSeries",
    description="A time series for a heat sensor",
    unit="degrees, celsius"
)
sensor_2_timeseries_external_id = sensor_instance_2_external_id + ":timeseries"
sensor_2_timeseries = TimeSeries(
    external_id=sensor_2_timeseries_external_id,
    asset_id=sensor_instance_2_id,
    name="heatSensorTimeSeries",
    description="A time series for a heat sensor",
    unit="degrees, celsius"
)
client.time_series.create([sensor_1_timeseries, sensor_2_timeseries])

,id,external_id,name,is_string,unit,asset_id,is_step,description,created_time,last_updated_time
0,8743103404932682,heatSensor:910db54d-dde6-452a-8cf0-8c7e887a28e...,heatSensorTimeSeries,False,"degrees, celsius",7418793056223224,False,A time series for a heat sensor,1586183558050,1586183558050
1,5695831270913089,heatSensor:3b9eb1d1-1e49-41d3-8e97-8dcd5f6d4ae...,heatSensorTimeSeries,False,"degrees, celsius",6842794385662259,False,A time series for a heat sensor,1586183558050,1586183558050


In [35]:
sensor_break_time = int(datetime(2017, 10, 15, 14, 35, 47).timestamp() * 1000)
sensor_replacement_time = int(datetime(2017, 10, 15, 14, 41, 42).timestamp() * 1000)

sensor_1_start = sensor_break_time - (10 * 1000)
sensor_1_end = sensor_replacement_time + (10 * 1000)
sensor_1_data = lambda x: random.uniform(50, 55) if x < sensor_break_time else random.uniform(100,200)
sensor_1_datapoints = [(x, sensor_1_data(x)) for x in range(sensor_1_start, sensor_1_end, 1000)] 

sensor_2_start = sensor_replacement_time
sensor_2_end = sensor_replacement_time + (30 * 1000)
sensor_2_data = lambda x: random.uniform(50,55)
sensor_2_datapoints = [(x, sensor_2_data(x)) for x in range(sensor_2_start, sensor_2_end, 1000)]

client.datapoints.insert(sensor_1_datapoints, external_id=sensor_1_timeseries_external_id)
client.datapoints.insert(sensor_2_datapoints, external_id=sensor_2_timeseries_external_id)

In [37]:
widget_heat_sensor_resource = {"resource": "asset", "resourceId": widget_heat_sensor_external_id}
sensor_1_resource = {"resource": "asset", "resourceId": sensor_instance_1_external_id}
sensor_2_resource = {"resource": "asset", "resourceId": sensor_instance_2_external_id}
sensor_1_timeseries_resource = {"resource": "timeseries", "resourceId": sensor_1_timeseries_external_id}
sensor_2_timeseries_resource = {"resource": "timeseries", "resourceId": sensor_2_timeseries_external_id}

sensor_1_to_timeseries_relationship_external_id = sensor_instance_1_external_id + ":readings"
sensor_1_to_timeseries_relationship = Relationship(
    external_id=sensor_1_to_timeseries_relationship_external_id,
    relationship_type="belongsTo",
    confidence="0.99",
    source=sensor_1_timeseries_resource,
    target=sensor_1_resource,
    data_set=source
)
sensor_2_to_timeseries_relationship_external_id = sensor_instance_2_external_id + ":readings"
sensor_2_to_timeseries_relationship = Relationship(
    external_id=sensor_2_to_timeseries_relationship_external_id,
    relationship_type="belongsTo",
    confidence="0.99",
    source=sensor_2_timeseries_resource,
    target=sensor_2_resource,
    data_set=source
)
widget_heat_sensor_implementation_1_external_id = widget_heat_sensor_external_id + ":impl:1"
widget_heat_sensor_1_relationship = Relationship(
    external_id=widget_heat_sensor_implementation_1_external_id,
    relationship_type="implements",
    confidence="0.99",
    source=sensor_1_resource,
    target=widget_heat_sensor_resource,
    data_set=source,
    end_time = sensor_break_time
)
widget_heat_sensor_implementation_2_external_id = widget_heat_sensor_external_id + ":impl:2"
widget_heat_sensor_2_relationship = Relationship(
    external_id=widget_heat_sensor_implementation_2_external_id,
    relationship_type="implements",
    confidence="0.99",
    source=sensor_2_resource,
    target=widget_heat_sensor_resource,
    data_set=source,
    start_time = sensor_replacement_time
)
relationships = [sensor_1_to_timeseries_relationship, sensor_2_to_timeseries_relationship, widget_heat_sensor_1_relationship, widget_heat_sensor_2_relationship]
client.relationships.create(relationships)

,source,target,confidence,data_set,external_id,relationship_type,created_time,last_updated_time,end_time,start_time
0,{'resourceId': 'heatSensor:910db54d-dde6-452a-...,{'resourceId': 'heatSensor:910db54d-dde6-452a-...,0.99,time_stitching_demo,heatSensor:910db54d-dde6-452a-8cf0-8c7e887a28e...,belongsTo,1586183575118,1586183575118,<NA>,<NA>
1,{'resourceId': 'heatSensor:3b9eb1d1-1e49-41d3-...,{'resourceId': 'heatSensor:3b9eb1d1-1e49-41d3-...,0.99,time_stitching_demo,heatSensor:3b9eb1d1-1e49-41d3-8e97-8dcd5f6d4ae...,belongsTo,1586183575118,1586183575118,<NA>,<NA>
2,{'resourceId': 'heatSensor:910db54d-dde6-452a-...,{'resourceId': 'widgetHeatSensor:229fd248-51b8...,0.99,time_stitching_demo,widgetHeatSensor:229fd248-51b8-40a5-9f28-0793b...,implements,1586183575118,1586183575118,1508078147000,<NA>
3,{'resourceId': 'heatSensor:3b9eb1d1-1e49-41d3-...,{'resourceId': 'widgetHeatSensor:229fd248-51b8...,0.99,time_stitching_demo,widgetHeatSensor:229fd248-51b8-40a5-9f28-0793b...,implements,1586183575118,1586183575118,<NA>,1508078502000


In [38]:
heat_sensors = client.assets.list(external_id_prefix="widgetHeatSensor")
heat_sensors

,external_id,name,description,metadata,source,id,created_time,last_updated_time,root_id
0,widgetHeatSensor:229fd248-51b8-40a5-9f28-0793b...,widgetHeatSensor,A heat sensor for a widget,{},time_stitching_demo,7736683234368567,1586183545674,1586183545674,7736683234368567


In [39]:
sensors = client.relationships.list(
    relationship_type="implements",
    targets = [{"resourceId": heat_sensors[0].external_id}]).to_pandas()
sensors["sensorId"] = sensors["source"].map(lambda source: str(source["resourceId"]))
sensor_filter_data = sensors[["sensorId", "startTime", "endTime"]]
sensor_filter_data

,source,target,startTime,confidence,dataSet,externalId,relationshipType,createdTime,lastUpdatedTime,endTime,sensorId
0,{'resourceId': 'heatSensor:3b9eb1d1-1e49-41d3-...,{'resourceId': 'widgetHeatSensor:229fd248-51b8...,1508078502000,0.99,time_stitching_demo,widgetHeatSensor:229fd248-51b8-40a5-9f28-0793b...,implements,1586183575118,1586183575118,<NA>,heatSensor:3b9eb1d1-1e49-41d3-8e97-8dcd5f6d4aee
1,{'resourceId': 'heatSensor:910db54d-dde6-452a-...,{'resourceId': 'widgetHeatSensor:229fd248-51b8...,<NA>,0.99,time_stitching_demo,widgetHeatSensor:229fd248-51b8-40a5-9f28-0793b...,implements,1586183575118,1586183575118,1508078147000,heatSensor:910db54d-dde6-452a-8cf0-8c7e887a28e2


In [41]:
matching_sensor_ids = sensor_filter_data["sensorId"].tolist()
matching_sensor_ids

['heatSensor:3b9eb1d1-1e49-41d3-8e97-8dcd5f6d4aee',
 'heatSensor:910db54d-dde6-452a-8cf0-8c7e887a28e2']

In [42]:
matching_sensor_ids = sensor_filter_data["sensorId"].tolist()
sensor_timeseries_relationships = client.relationships.list(
    sources=[{"resource": "timeseries"}],
    targets = [{"resourceId" : ms} for ms in matching_sensor_ids]).to_pandas()
sensor_timeseries_source_target = sensor_timeseries_relationships[["target", "source"]]
sensor_timeseries_external_ids = sensor_timeseries_source_target.applymap(lambda x: str(x["resourceId"]))

sensor_timeseries_column_renaming = { "source": "timeseriesId", "target": "sensorId" }
sensor_timeseries = sensor_timeseries_external_ids.rename(colums=sensor_timeseries_column_renaming)

,source,target,confidence,dataSet,externalId,relationshipType,createdTime,lastUpdatedTime
0,{'resourceId': 'heatSensor:3b9eb1d1-1e49-41d3-...,{'resourceId': 'heatSensor:3b9eb1d1-1e49-41d3-...,0.99,time_stitching_demo,heatSensor:3b9eb1d1-1e49-41d3-8e97-8dcd5f6d4ae...,belongsTo,1586183575118,1586183575118
1,{'resourceId': 'heatSensor:910db54d-dde6-452a-...,{'resourceId': 'heatSensor:910db54d-dde6-452a-...,0.99,time_stitching_demo,heatSensor:910db54d-dde6-452a-8cf0-8c7e887a28e...,belongsTo,1586183575118,1586183575118


In [44]:
datapoints_filter_dataframe = sensor_filter_data.merge(sensor_timeseries, how="inner", on="sensorId")
normalize_start = lambda x: x if isinstance(x,int) else 0
datapoints_filter_dataframe["startTime"] = datapoints_filter_dataframe["startTime"].apply(normalize_start)
now = int(datetime.now().timestamp()) * 1000
normalize_end = lambda x: x if isinstance(x,int) else now
datapoints_filter_dataframe["endTime"] = datapoints_filter_dataframe["endTime"].apply(normalize_end)
datapoints_filters = datapoints_filter_dataframe.to_dict(orient="records")
datapoints_filters.sort(key=lambda x: x["startTime"])
datapoints_filters

,sensorId,startTime,endTime,timeseriesId
0,heatSensor:3b9eb1d1-1e49-41d3-8e97-8dcd5f6d4aee,1508078502000,1586183644000,heatSensor:3b9eb1d1-1e49-41d3-8e97-8dcd5f6d4ae...
1,heatSensor:910db54d-dde6-452a-8cf0-8c7e887a28e2,0,1508078147000,heatSensor:910db54d-dde6-452a-8cf0-8c7e887a28e...


In [46]:
datapoints = [
    client.datapoints.retrieve(
        external_id = fd["timeseriesId"],
        start = fd["startTime"],
        end = fd["endTime"]).to_pandas().rename(
        columns={ fd["timeseriesId"]: "data" }) for fd in datapoints_filters
    ]
final_dataframe = datapoints[0].append(datapoints[1])
final_dataframe

[                          data
 2017-10-15 14:35:37  53.354358
 2017-10-15 14:35:38  54.481300
 2017-10-15 14:35:39  54.900856
 2017-10-15 14:35:40  54.551472
 2017-10-15 14:35:41  50.777159
 2017-10-15 14:35:42  50.984917
 2017-10-15 14:35:43  50.590293
 2017-10-15 14:35:44  50.879683
 2017-10-15 14:35:45  52.444897
 2017-10-15 14:35:46  52.808611,
                           data
 2017-10-15 14:41:42  53.145027
 2017-10-15 14:41:43  54.399578
 2017-10-15 14:41:44  54.538776
 2017-10-15 14:41:45  52.667667
 2017-10-15 14:41:46  53.154130
 2017-10-15 14:41:47  54.223637
 2017-10-15 14:41:48  54.794131
 2017-10-15 14:41:49  50.451333
 2017-10-15 14:41:50  51.517690
 2017-10-15 14:41:51  53.516740]

In [48]:
relationship_external_ids = [
    sensor_1_to_timeseries_relationship_external_id,
    sensor_2_to_timeseries_relationship_external_id,
    widget_heat_sensor_implementation_1_external_id,
    widget_heat_sensor_implementation_2_external_id
]
client.relationships.delete(relationship_external_ids)

timeseries_external_ids = [
    sensor_1_timeseries_external_id,
    sensor_2_timeseries_external_id
]
for timeseries_external_id in timeseries_external_ids:
    client.datapoints.delete_range(start=0, end=now, external_id=timeseries_external_id)
    client.time_series.delete(external_id=timeseries_external_id)
    
asset_external_ids = [
    widget_heat_sensor_external_id,
    sensor_instance_1_external_id,
    sensor_instance_2_external_id
]
client.assets.delete(external_id=asset_external_ids)